<a href="https://colab.research.google.com/github/vinayakShenoy/DL4CV/blob/master/Code/preprocessing/preprocessors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
import os
from tensorflow.keras.preprocessing.image import img_to_array
import imutils
from sklearn.feature_extraction.image import extract_patches_2d

In [ ]:
class SimplePreprocessor:
    def __init__(self, width, height, inter=cv2.INTER_AREA):
        self.width = width
        self.height = height
        self.inter = inter
        
    def preprocess(self, image):
        return cv2.resize(image, (self.width, self.height), 
                         interpolation=self.inter)

In [ ]:
class SimpleDatasetLoader:
    def __init__(self, preprocessors=None):
        self.preprocessors = preprocessors
        if self.preprocessors is None:
            self.preprocessors = []
    
    def load(self, imagePaths, verbose=-1):
        data = []
        labels = []
        for (i, imagePath) in enumerate(imagePaths):
            image = cv2.imread(imagePath)
            label = imagePath.split(os.path.sep)[-2]
            if self.preprocessors is not None:
                for p in self.preprocessors:
                    image = p.preprocess(image)
            data.append(image)
            labels.append(label)
            if verbose>0 and i>0 and (i+1)%verbose==0:
                print("[INFO] processed {}/{}".format(i+1, len(imagePaths)))
        return (np.array(data), np.array(labels))

In [ ]:
class ImageToArrayPreprocessor:
    def __init__(self, dataFormat=None):
        self.dataFormat = dataFormat
        
    def preprocess(self, image):
        return img_to_array(image, data_format=self.dataFormat)

In [ ]:
class AspectAwarePreprocessor:
    def __init__(self, width, height, inter=cv2.INTER_AREA):
        self.width = width
        self.height = height
        self.inter = inter
  
    def preprocess(self, image):
        (h, w) = image.shape[:2]
        dW = 0
        dH = 0
        if w<h:
            image = imutils.resize(image, width=self.width, inter=self.inter)
              dH = int((image.shape[0] - self.height)/2.0)
        else:
            image = imutils.resize(image, height=self.height, inter=self.inter)
            dW = int((image.shape[1] - self.width)/2.0)

        (h,w) = image.shape[:2]
        image = image[dH:h-dH, dW:w-dW]
        return cv2.resize(image, (self.width, self.height), interpolation=self.inter)

In [ ]:
# A mean subtraction pre-processor designed to subtract the 
# mean RGB intensities across a dataset from a input image.
class MeanPreprocessor:
    def __init__(self, rMean, gMean, bMean):
        self.rMean = rMean
        self.gMean = gMean
        self.bMean = bMean
    
    def preprocess(self, image):
        (B,G,R) = cv2.split(image.astype("float32"))
        
        R -= self.rMean
        G -= self.gMean
        B -= self.bMean
        
        return cv2.merge([B, G, R])

In [ ]:
# A patch preprocessor used to randomly extract M × N pixel regions from an image during
# training.
class PatchPreprocessor:
    def __init__(self, width, height):
        self.width = width
        self.height = height
        
    def preprocess(self, image):
        return extract_patches_2d(image, (self.height, self.width),
                                 max_patches=1)[0]

In [ ]:
# An over-sampling pre-processor used at testing time to sample five regions of an input image
# (the four corners + center area) along with their corresponding horizontal flips
class CropPreprocessor:
    def __init__(self, width, height, horiz=True, inter=cv2.INTER_AREA):
        self.width = width
        self.height = height
        self.horiz = horiz
        self.inter = inter
    
    def preprocess(self, image):
        crops = []
        
        (h,w) = image.shape[:2]
        coords = [
            [0,0,self.width, self.height],
            [w - self.width, 0, w, self.height],
            [w - self.width, h - self.height, w, h],
            [0, h-self.height, self.width, h]
        ]
        
        dW = int(0.5*(w-self.width))
        dH = int(0.5*(h-self.height))
        coords.append([dW, dH, w-dW, h-dH])
        
        for (startX, startY, endX, endY) in coords:
            crop = image[startY:endY, startX:endX]
            crop = cv2.resize(crop, (self.width, self.height),
                             interpolation=self.inter)
            crop.append(crop)
        
        if self.horiz:
            mirrors = [cv2.flip(c, 1) for c in crops]
            crops.extend(mirrrors)
        
        return np.array(crops)